In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from newspaper import Article
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs
import datetime
import csv
import sys
import schedule
import os

def Crawling() :

    url='https://www.newsnow.co.uk/h/Industry+Sectors/Commodities/Industrial+Metals/Nickel?type=ln'

    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    
    
    driver_path = '/usr/bin/chromedriver'
    driver=webdriver.Chrome(driver_path, options=options)
    driver.implicitly_wait(3)
    driver.get(url)

    res = requests.get(url)
    soup = bs(res.text, 'lxml')
    div = soup.select_one("div.rs-grid.rs-grid--skeleton.js-skeleton > main > div.rs-newsbox.js-newsbox.js-newsmain.js-central_ln_wrap > div").findAll('div', {"class" : "hl"})

    temp_link_list = []
    link_list = []
    name_list = []
    article_list = []
    date_list = []


    for hl in div :
        data = hl.find('a', {"class" : "hll"})
        time = hl.find('span', {'class' : "time"})
        
        date = time.text
        date = pd.to_datetime(date)
        date = date.strftime("%Y-%m-%d %H:%M:%S")
        date_list.append(date)

        temp_link_list.append(data.get('href'))
        name_list.append(data.text)


    for url in temp_link_list :
        res = requests.get(url)
        soup = bs(res.text, 'lxml')
        url = soup.select_one('script').get_text().split(" ")[13]
        urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', str(url))
        urls = ''.join(urls)
        link_list.append(urls)


    for url in link_list :
        article = Article(url)
        try :
            article.download()
            article.parse()
            if len(article.text) > 30000 :
                article_list.append(article.summary)
            else :
                article_list.append(article.text)
        except :
            article_list.append('')


    df = pd.DataFrame({'link' : link_list,
                       'title': name_list,
                       'time': date_list,
                       'article': article_list
                      })
    

    return df

In [ ]:
df = Crawling()

In [ ]:
df.sort_index(axis=0, ascending=False, ignore_index=True, inplace=True)

In [ ]:
df['link'] = df['link'].astype('string')
df['title'] = df['title'].astype('string')
df['time'] = df['time'].astype('string')
df['article'] = df['article'].astype('string')

In [ ]:
df

In [ ]:
df.to_csv('news_data_nickel.csv',index=False)